In [7]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

def extract_pose_features(image_path):
    image = cv2.imread(image_path)
    
    alto_original, ancho_original = image.shape[:2]
    nuevo_alto = 700
    nuevo_ancho = int((nuevo_alto / alto_original) * ancho_original)
    image = cv2.resize(image, (nuevo_ancho, nuevo_alto))

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        keypoints = [(lm.x, lm.y, lm.z) for lm in landmarks]
        
        left_shoulder = keypoints[11]
        right_shoulder = keypoints[12]
        left_elbow = keypoints[13]
        right_elbow = keypoints[14]
        left_wrist = keypoints[15]
        right_wrist = keypoints[16]
        left_hip = keypoints[23]
        right_hip = keypoints[24]
        left_knee = keypoints[25]
        right_knee = keypoints[26]
        left_ankle = keypoints[27]
        right_ankle = keypoints[28]

        left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
        right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        
        features = np.array(keypoints).flatten().tolist()
        features += [left_elbow_angle, right_elbow_angle, left_shoulder_angle, right_shoulder_angle, left_knee_angle, right_knee_angle]
        return features
    return []

def process_images(data_dir):
    all_data = []
    all_labels = []

    for category in os.listdir(data_dir):
        category_path = os.path.join(data_dir, category)
        if os.path.isdir(category_path):
            for image in os.listdir(category_path):
                image_path = os.path.join(category_path, image)
                if image_path.endswith(('.jpg', '.png', '.jpeg')):
                    print(f"Procesando imagen: {image_path}")
                    data = extract_pose_features(image_path)
                    if len(data) > 0:  # Verifica que haya datos para agregar
                        all_data.append(data)
                        all_labels.append(category)

    return all_data, all_labels

def save_to_csv(data, labels, output_file):
    num_keypoints = 33 * 3
    columns = [f'keypoint_{i}' for i in range(num_keypoints)]
    columns += ['left_elbow_angle', 'right_elbow_angle', 'left_shoulder_angle', 'right_shoulder_angle', 'left_knee_angle', 'right_knee_angle']
    
    df = pd.DataFrame(data, columns=columns)
    df['label'] = labels
    df.to_csv(output_file, index=False)

# Definir rutas
ruta_actual = os.getcwd()
carpeta_mediapipe = os.path.dirname(ruta_actual)
ruta_raiz = os.path.dirname(carpeta_mediapipe)

ruta_imagenes = os.path.join(ruta_raiz, 'Data Set', 'images')
ruta_csv = os.path.join(carpeta_mediapipe, 'Datos_Pose', 'pose_data_images.csv')

# Procesar imágenes y guardar a CSV
image_features, image_labels = process_images(ruta_imagenes)
save_to_csv(image_features, image_labels, ruta_csv)


Procesando imagen: c:\Users\Diego Castillo\Desktop\TFM\ComparativaModelosIA_TFM\Data Set\images\curl de biceps\barbell biceps curl_1000001.jpg


c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Procesando imagen: c:\Users\Diego Castillo\Desktop\TFM\ComparativaModelosIA_TFM\Data Set\images\curl de biceps\barbell biceps curl_100001.jpg
Procesando imagen: c:\Users\Diego Castillo\Desktop\TFM\ComparativaModelosIA_TFM\Data Set\images\curl de biceps\barbell biceps curl_1000011.jpg
Procesando imagen: c:\Users\Diego Castillo\Desktop\TFM\ComparativaModelosIA_TFM\Data Set\images\curl de biceps\barbell biceps curl_1000021.jpg
Procesando imagen: c:\Users\Diego Castillo\Desktop\TFM\ComparativaModelosIA_TFM\Data Set\images\curl de biceps\barbell biceps curl_1000031.jpg
Procesando imagen: c:\Users\Diego Castillo\Desktop\TFM\ComparativaModelosIA_TFM\Data Set\images\curl de biceps\barbell biceps curl_1000041.jpg
Procesando imagen: c:\Users\Diego Castillo\Desktop\TFM\ComparativaModelosIA_TFM\Data Set\images\curl de biceps\barbell biceps curl_1000051.jpg
Procesando imagen: c:\Users\Diego Castillo\Desktop\TFM\ComparativaModelosIA_TFM\Data Set\images\curl de biceps\barbell biceps curl_1000061.jpg
